In [3]:
!tune download mistralai/Mistral-7B-v0.1\
    --hf-token <hf token>

/opt/conda/envs/py_3.12/lib/python3.12/site-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
  warnings.warn(msg)
Ignoring files matching the following patterns: None
.gitattributes: 100%|██████████████████████| 1.52k/1.52k [00:00<00:00, 18.8MB/s]
README.md: 100%|███████████████████████████| 1.55k/1.55k [00:00<00:00, 20.9MB/s]
generation_config.json: 100%|██████████████████| 116/116 [00:00<00:00, 1.35MB/s]
model-00001-of-00002.safetensors:  53%|██▋  | 5.26G/9.94G [00:07<00:05, 789MB/s]/opt/conda/envs/py_3.12/lib/python3.12/site-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
  warnings.warn(msg)
model-00001-of-00002.safetensors: 100%|█████| 9.94G/9.94G [00:13<00:00, 726MB/s]
model-00002-of-00002.safetensors: 100%|████| 4.54G/4.54G [00:03<00:00, 1.41GB/s]
model.safetensors.index.json: 100%|████████| 25.1k/25.1k [00:00<00:00, 11.2MB/s]
pytorch_model-00001-of-00002.bin: 100

In [4]:
!tune download google/gemma-2-2b-it\
    --hf-token <hf token>

/opt/conda/envs/py_3.12/lib/python3.12/site-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
  warnings.warn(msg)
Ignoring files matching the following patterns: None
Successfully downloaded model repo and wrote to the following locations:
/tmp/gemma-2-2b-it/README.md
/tmp/gemma-2-2b-it/generation_config.json
/tmp/gemma-2-2b-it/.gitattributes
/tmp/gemma-2-2b-it/tokenizer_config.json
/tmp/gemma-2-2b-it/tokenizer.json
/tmp/gemma-2-2b-it/.cache
/tmp/gemma-2-2b-it/special_tokens_map.json
/tmp/gemma-2-2b-it/model-00001-of-00002.safetensors
/tmp/gemma-2-2b-it/original_repo_id.json
/tmp/gemma-2-2b-it/model-00002-of-00002.safetensors
/tmp/gemma-2-2b-it/config.json
/tmp/gemma-2-2b-it/model.safetensors.index.json
/tmp/gemma-2-2b-it/tokenizer.model


In [5]:
!tune cat gemma/2B_lora 


/opt/conda/envs/py_3.12/lib/python3.12/site-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
  warnings.warn(msg)
output_dir: /tmp/torchtune/gemma_2B/lora
tokenizer:
    _component_: torchtune.models.gemma.gemma_tokenizer
    path: /tmp/gemma-2b/tokenizer.model
dataset:
    _component_: torchtune.datasets.alpaca_dataset
    packed: false
seed: null
shuffle: true
model:
    _component_: torchtune.models.gemma.lora_gemma_2b
    lora_attn_modules:
    - q_proj
    - v_proj
    - output_proj
    apply_lora_to_mlp: true
    lora_rank: 64
    lora_alpha: 128
    lora_dropout: 0.0
checkpointer:
    _component_: torchtune.training.FullModelHFCheckpointer
    checkpoint_dir: /tmp/gemma-2b/
    checkpoint_files:
    - model-00001-of-00002.safetensors
    - model-00002-of-00002.safetensors
    recipe_checkpoint: null
    output_dir: ${output_dir}
    model_type: GEMMA
resume_from_checkpoint: false
save_adapter_weights_only: false
optimizer

In [53]:
#loading from dataset fri data.json sharegpt formate 
from torchtune.models.mistral import mistral_tokenizer
from torchtune.datasets import chat_dataset

m_tokenizer = mistral_tokenizer(
    path="/tmp/Mistral-7B-v0.1/tokenizer.model",
    prompt_template="torchtune.models.mistral.MistralChatTemplate",
    max_seq_len=8192,
)
ds = chat_dataset(
    tokenizer=m_tokenizer,
    source="json",
    data_files="data/chat_dataset.json",
    split="train",
    conversation_column="conversations",
    conversation_style="sharegpt",
    # By default, user prompt is ignored in loss. Set to True to include it
    train_on_input=True,
    new_system_prompt=None,
)
tokenized_dict = ds[0]
tokens, labels = tokenized_dict["tokens"], tokenized_dict["labels"]
print(m_tokenizer.decode(tokens))
# [INST] What is the answer to life?  [/INST] The answer is 42. [INST] That's ridiculous  [/INST] Oh I know.
print(labels)
# [1, 733, 16289, 28793, 1824, 349, 272, 4372, ...]

Generating train split: 0 examples [00:00, ? examples/s]

[INST] What is the answer to life?  [/INST] The answer is 42. [INST] That's ridiculous  [/INST] Oh I know.
[1, 733, 16289, 28793, 1824, 349, 272, 4372, 298, 1411, 28804, 28705, 733, 28748, 16289, 28793, 415, 4372, 349, 28705, 28781, 28750, 28723, 2, 1, 733, 16289, 28793, 1725, 28742, 28713, 16706, 28705, 733, 28748, 16289, 28793, 5434, 315, 873, 28723, 2]


In [54]:
#Loading  data from hugging face 
from torchtune.models.gemma import gemma_tokenizer
from torchtune.datasets import chat_dataset

g_tokenizer = gemma_tokenizer("/tmp/gemma-2-2b-it/tokenizer.model")
ds = chat_dataset(
    tokenizer=g_tokenizer,
    source="Open-Orca/SlimOrca-Dedup",
    conversation_column="conversations",
    conversation_style="sharegpt",
    split="train",
)

In [43]:
g_tokenizer.decode(ds[0]["tokens"])

'You are an AI assistant. You will be given a task. You must generate a detailed and long answer.Write an article based on this "A man has been charged with murder and attempted murder after a woman and the man she was on a date with were stabbed at a restaurant in Sydney, Australia."Title: Tragedy Strikes in Sydney: Victims Stabbed during Dinner Date, Suspect Charged with Murder and Attempted Murder\n\nIntroduction\n\nA seemingly ordinary evening took a tragic turn in Sydney, Australia when a romantic dinner date at a local restaurant ended in bloodshed, as a man allegedly attacked a couple, stabbing both victims. Local authorities have charged the suspect with murder and attempted murder in connection with the horrific crime. In this article, we delve into the chilling incident, the suspect, and the impact it has had on the community.\n\nThe Incident\n\nThe alarming event took place at a popular restaurant in Sydney, Australia. A woman and her date were enjoying their meal – blissful

In [55]:
from torchtune.models.gemma import gemma_tokenizer
from torchtune.datasets import instruct_dataset

g_tokenizer = gemma_tokenizer(
    path="/tmp/gemma-2-2b-it/tokenizer.model",
    prompt_template="torchtune.data.GrammarErrorCorrectionTemplate",
    max_seq_len=8192,
)
ds = instruct_dataset(
    tokenizer=g_tokenizer,
    source="csv",
    data_files="data/instruct_dataset.csv",
    split="train",
    # By default, user prompt is ignored in loss. Set to True to include it
    train_on_input=True,
    # Prepend a system message to every sample
    new_system_prompt="You are an AI assistant. ",
    # Use columns in our dataset instead of default
    column_map={"input": "incorrect", "output": "correct"},
)
tokenized_dict = ds[0]
tokens, labels = tokenized_dict["tokens"], tokenized_dict["labels"]
print(g_tokenizer.decode(tokens))
# You are an AI assistant. Correct this to standard English:This are a cat---\nCorrected:This is a cat.
print(labels)  # System message is masked out, but not user message
# [-100, -100, -100, -100, -100, -100, 27957, 736, 577, ...]

Generating train split: 0 examples [00:00, ? examples/s]

You are an AI assistant.Correct this to standard English:This are a cat
---
Corrected:This is a cat.
[-100, -100, -100, -100, -100, -100, -100, 27957, 736, 577, 5039, 4645, 235292, 1596, 708, 476, 4401, 108, 3976, 108, 198286, 235292, 1596, 603, 476, 4401, 235265, 1]


In [56]:
tokenized_dict = ds[1]
tokens, labels = tokenized_dict["tokens"], tokenized_dict["labels"]
print(g_tokenizer.decode(tokens))
# You are an AI assistant. Correct this to standard English:This are a cat---\nCorrected:This is a cat.
print(labels)  # System message is masked out, but not user message
# [-100, -100, -100, -100, -100, -100, 27957, 736, 577, ...]

You are an AI assistant.Correct this to standard English:Helio
---
Corrected: hello
[-100, -100, -100, -100, -100, -100, -100, 27957, 736, 577, 5039, 4645, 235292, 1949, 4167, 108, 3976, 108, 198286, 235292, 25612, 1]


In [63]:
# In code
from torchtune.models.gemma import gemma_tokenizer
from torchtune.datasets import instruct_dataset

g_tokenizer = gemma_tokenizer("/tmp/gemma-2-2b/tokenizer.model")
ds = instruct_dataset(
    tokenizer=g_tokenizer,
    source="json",
    data_files="data/instruct_data.json",
    column_map={"input": "prompt", "output": "response"},
)

Generating train split: 0 examples [00:00, ? examples/s]

In [66]:
g_tokenizer.decode(ds[0]['tokens'])

'hello worldbye world'

In [68]:
# In code
from torchtune.models.gemma import gemma_tokenizer
from torchtune.datasets import instruct_dataset

g_tokenizer = gemma_tokenizer("/tmp/gemma-2-2b/tokenizer.model")
ds = instruct_dataset(
    tokenizer=g_tokenizer,
    source="json",
    data_files="data/database.json",
    column_map={"input": "input", "output": "output"},)

Generating train split: 0 examples [00:00, ? examples/s]

In [72]:
g_tokenizer.decode(ds[0]['tokens'])

'Which customers from Pune have placed an order in the last month?To answer this, you need to query the `customers` and `orders` tables.'

In [8]:
!tune download meta-llama/Meta-Llama-3-8B-Instruct\
    --hf-token <hf token>

/opt/conda/envs/py_3.12/lib/python3.12/site-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
  warnings.warn(msg)
Ignoring files matching the following patterns: None
.gitattributes: 100%|██████████████████████| 1.52k/1.52k [00:00<00:00, 21.6MB/s]
LICENSE: 100%|█████████████████████████████| 7.80k/7.80k [00:00<00:00, 74.9MB/s]
README.md: 100%|███████████████████████████| 38.9k/38.9k [00:00<00:00, 5.09MB/s]
USE_POLICY.md: 100%|███████████████████████| 4.70k/4.70k [00:00<00:00, 62.3MB/s]
generation_config.json: 100%|██████████████████| 187/187 [00:00<00:00, 2.86MB/s]
model-00001-of-00004.safetensors: 100%|█████| 4.98G/4.98G [00:06<00:00, 732MB/s]
model-00002-of-00004.safetensors:   0%|             | 0.00/5.00G [00:00<?, ?B/s]/opt/conda/envs/py_3.12/lib/python3.12/site-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
  warnings.warn(msg)
model-00002-of-00004.safetensors: 100

In [9]:
# In code
from torchtune.models.llama3_2_vision import llama3_2_vision_transform
from torchtune.datasets.multimodal import multimodal_chat_dataset

model_transform = llama3_2_vision_transform(
    path="/tmp/Meta-Llama-3-8B-Instruct/original/tokenizer.model",
    max_seq_len=8192,
    image_size=560,
)
ds = multimodal_chat_dataset(
    model_transform=model_transform,
    source="Lin-Chen/ShareGPT4V",
    split="train",
    name="ShareGPT4V",
    image_dir="/home/user/dataset/",
    image_tag="<image>",
)

README.md: 0.00B [00:00, ?B/s]

(…)egpt4v_instruct_gpt4-vision_cap100k.json:   0%|          | 0.00/134M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/102025 [00:00<?, ? examples/s]